In [1]:
#! /usr/bin/env python

"""
Simulation to analyze mating dynamics of cowbirds (or more generally, monogamy in the absence of parental care) 
written by Ammon Perkes (perkes.ammon@gmail.com) at University of Pennsylvania, 2016
""" 

import sys, os
import numpy as np
from  matplotlib import pyplot as plt
from matplotlib.widgets import Slider
import subprocess # Only used for making a sound to let you know when it's done
import plotly as py
import networkx as nx               # Necessary for all the network analysis and plotting
#from plotly.graph_objs import *    # For playing with nice network graphs, not necessary

#External .py dependencies, contatining Strategies and Stats
import SimStrategies,SimStats

# For Debugging
from IPython.core.debugger import Tracer 
#Tracer()() 

# Default parameters. These can be defined invdividually as well through the parameter class. 

N_MALES = 6
N_FEMALES = 6
RES_M = 1.0       # Resource limitation for a male
RES_F = 1.0       # Resource limitation for a female
RESS_M = [RES_M] * N_MALES     # Resource limitation vector for all males
RESS_F = [RES_F] * N_FEMALES   # Resource limitation vector for all females
Q_MALE = 1.0      # Default Quality for a male
Q_FEMALE = 1.0    # Default Quality for a female
Q_MALES = [Q_MALE] * N_MALES     # Quality vector for maless
Q_FEMALES = [Q_FEMALE] * N_FEMALES # Quality vector for females 
TURNS = 1000       # Number of turns per trial
TRIALS = 10        # Number of trials per simulation
STRAT_M = 'M0'        # Default strategy for males
STRAT_F = 'F0'        # Default strategy for females 
STRATS_M = [STRAT_M] * N_MALES       # Strategy vector for males
STRATS_F = [STRAT_F] * N_FEMALES     # Strategy vector for females
ALPHA = 0             # Alpha scales the adjacency 
ALPHA2 = 1.5          # Scales the relative adjacency
ALPHAS = [ALPHA2] * N_FEMALES         # Alpha vector for females (currently defunct)
BETA = 1.5         # scales male investment
BETA2 = 0          # scales relative male investment
GAMMA = 1.5        # scales female investment
GAMMA2 = 0         # scales relative female investment
DELTA = 1.0        # scales male quality (^)
EPSILON = 1.0      # scales female quality (3)
ZETA = 1.0         # scales the (1-cost)
IOTA = 1.0
KAPPA = 0.01       # defines adjacency kost, think of it as risk of disease transmission
OMEGA = 0.5        # maximum cost of crowding
PHI = 0.0          # Fleeing multiplier when determining adjacency
RHO = 0.0          # base risk of individual association
TAU = 2.0          # scales the rate at which crowding cost increases
SHIFT = .1         # amount by which investment is shifted
DECAY = .01         # decay constant 
BUMP = .01         # random encounter rate
MIN_ADJ = .01      # Minimum adjacency
MAX_ADJ = 1.0      # Maximum adjacency
MIN_INV = .001    # Minimum investment (keeps everything connected)

## Functions that define how matrices covert, these are two of the central mechanisms of the simulation
def adjacency_to_reward(history, turn, params):
    ## This provides males and females with a reward function on which to base decisions
    # Noteably, this is different from the original implementation, in which the reward *is* the female response
    # Because it needs to update immediately (and really everything should...) it needs the current turn also
    # Should adjacency be symmetric? It doesn't make sense for it to not be symmetric 
    alpha_m, alpha_f = (params.alpha_m, params.alpha_f) # Adjacency benefit multiplier. 
    alpha2_m, alpha2_f = (params.alpha2_m, params.alpha2_f) # Relative adjacency benefit multiplier
    beta_m, beta_f = (params.beta_m, params.beta_f) # Investment benefit multiplier. 
    beta2_m, beta2_f = (params.beta2_m, params.beta2_f)
    gamma_m, gamma_f = (params.gamma_m, params.gamma_f)
    gamma2_m, gamma2_f = (params.gamma2_m, params.gamma2_f)
    delta_m, delta_f = (params.delta_m, params.delta_f)
    epsilon_m, epsilon_f = (params.epsilon_m, params.epsilon_f)
    zeta_m, zeta_f = (params.zeta_m, params.zeta_f)
    omega = params.omega ## This defines the maximum cost of crowding
    tau = params.tau  ## This defines how quickly crowding cost decreases
    kappa = params.kappa ## This defines base adjacency cost
    rho = params.rho ## This defines risk from individual (also a cost)
    n_birds = history.n_males + history.n_females
    r_function = params.r_function
    reward = np.zeros(np.shape(history.reward_matrix[0]))
    
    
    #current_adjacency = history.adjacency_matrix[history.current_turn]   # This way it gets the most recent adjacency
    current_adjacency = turn.adjacency # This way it gets the most recent adjacency
    current_investment = turn.invest
    homo_adjacency_cost = kappa  #
    het_adjacency_cost = kappa
    individual_cost = rho # This is an invariant risk of associating with any individual ()
    
    # There are different costs/benefits for males and females                  
    if r_function == 0:
        ## Calculate reward for males
        for m in range(history.n_males):
            i = m
            for f in range(history.n_females):         
                j = f + history.n_males
                crowding_cost = omega / (tau ** (n_birds - 1 - current_adjacency[:,j].sum() - current_adjacency[i,j]) + .0001)
                adjacency_cost = current_adjacency[i,j] * het_adjacency_cost + individual_cost
                adjacency_benefit = current_adjacency[i,j] ** alpha_m
                adjacency_benefit_rel = (current_adjacency[i,j] / (current_adjacency[:history.n_males,j].sum() + .0001)) ** alpha2_m
                m_investment = current_investment[i,j] ** beta_m
                m_investment_rel = (current_investment[i,j] / (current_investment[:history.n_males,j].sum() + .0001)) ** beta2_m
                f_investment = current_investment[j,i] ** gamma_m  
                f_investment_rel = (current_investment[j,i] / (current_investment[history.n_males:,i].sum() + .0001)) ** gamma2_m
                m_quality = history.quality_vector[i] ** delta_m
                f_quality = history.quality_vector[j] ** epsilon_m
                survival = (1 - crowding_cost - adjacency_cost) ** zeta_m
                reward[i,j] = (adjacency_benefit * adjacency_benefit_rel * m_investment * m_investment_rel * f_investment * f_investment_rel 
                               * f_quality * m_quality * survival)
                                   
            for m2 in range(history.n_males):
                adjacency_cost = current_adjacency[i,m2] * homo_adjacency_cost + individual_cost
                crowding_cost = omega / (tau ** (n_birds - current_adjacency[:,m2].sum()) + .0001)
                competition_cost = max(max(current_investment[m2,m],0) + min(current_investment[m,m2],0),0)
                cost = adjacency_cost + crowding_cost + competition_cost
                reward[i,m2] = - cost
            reward[i,i] = 0
        
        # Calculate reward for females
        for f in range(history.n_females):
            i = f + history.n_males
            for m in range(history.n_males):
                j = m
                crowding_cost = omega / (tau ** (n_birds - current_adjacency[:,j].sum()) + .0001)
                adjacency_cost = current_adjacency[i,j] * het_adjacency_cost + individual_cost
                adjacency_benefit = current_adjacency[i,j] ** alpha_f                
                adjacency_benefit_rel = (current_adjacency[i,j] / (current_adjacency[history.n_males:,j].sum() + .0001)) ** alpha2_f
                m_investment = current_investment[j,i] ** beta_f
                m_investment_rel = (current_investment[j,i] / (current_investment[:history.n_males,i].sum() + .0001)) ** beta2_f
                f_investment = current_investment[i,j] ** gamma_f  
                f_investment_rel = (current_investment[i,j] / (current_investment[history.n_males:,j].sum() + .0001)) ** gamma2_f
                m_quality = history.quality_vector[j] ** delta_f
                f_quality = history.quality_vector[i] ** epsilon_f
                survival = (1 - crowding_cost - adjacency_cost) ** zeta_f
                reward[i,j] = (adjacency_benefit * adjacency_benefit_rel * 
                               f_investment * f_investment_rel * m_investment * m_investment_rel 
                               * m_quality * f_quality * survival)
            for f2 in range(history.n_females):
                j = f2 + history.n_males
                adjacency_cost = current_adjacency[i,j] * homo_adjacency_cost + individual_cost
                crowding_cost = omega / (tau ** (n_birds - 1 - current_adjacency[:,j].sum() - current_adjacency[i,j]) + .0001)
                competition_cost = max(max(current_investment[j,i],0) + min(current_investment[i,j],0),0)
                cost = adjacency_cost + crowding_cost + competition_cost
                reward[i,j] = - cost
            reward[f,f] = 0   
    return reward

# Translate investment into adjacency
def investment_to_adjacency(history, params):
    # Here, investment cuts a fraction of the distance, using new investment matrix
    
    iota = params.iota ## This is a parameter that determines how investment scales
    phi = params.phi ## This is a parameter that determines how investment in fleeing scales (get it? phleeing)
    shift_constant = params.shift_constant ## This scales down investment to make everything more gradual
    decay_constant = params.decay_constant
    min_adjacency = params.min_adjacency
    max_adjacency = params.max_adjacency
    previous_adjacency = history.adjacency_matrix[history.current_turn - 1]
    a_function = params.a_function
    ### The following line adds slippage, this is important to avoid crowding when no one is fleeing.
    if a_function == 0:
        encounter_rate = params.bump_constant
        random_encounter = (1 - previous_adjacency) * encounter_rate
        previous_adjacency = previous_adjacency * (1 - decay_constant)

        previous_investment = history.invest_matrix[history.current_turn - 1]
        
        shift_matrix = np.zeros(np.shape(previous_investment))
        shift_matrix[previous_investment > 0] = previous_investment[previous_investment > 0] ** iota
        shift_matrix[previous_investment < 0] = - (phi + (1 - phi) * np.abs(previous_investment[previous_investment < 0]) ** iota) 
        
        current_adjacency = previous_adjacency + (previous_adjacency) * shift_matrix * shift_constant + random_encounter
        
        #Symetrize adjacency matrix
        current_adjacency = (current_adjacency + current_adjacency.T) / 2.
        current_adjacency[current_adjacency < min_adjacency] = min_adjacency
        current_adjacency[current_adjacency > max_adjacency] = max_adjacency
    return current_adjacency

# Parameter object containing all necessary parameters. 
class Parameters(object):
    def __init__(self, r_function = 0, a_function = 0, n_turns = TURNS, n_trials = TRIALS,
                 strat_f = STRAT_F, strat_m = STRAT_M, strats_f = None, strats_m = None, 
                 res_m = RES_M, res_f = RES_F, ress_m = None, ress_f = None,
                 n_males = N_MALES, n_females = N_FEMALES,
                 q_male = Q_MALE, q_female = Q_FEMALE, q_males = None, q_females = None,
                 alpha = ALPHA, alpha_m = None, alpha_f = None, alpha2 = ALPHA2, alpha2_m = None, alpha2_f = None, alphas = None, 
                 beta = BETA, beta2 = BETA2, beta_m = None, beta_f = None, beta2_m = None, beta2_f = None,
                 gamma = GAMMA, gamma2 = GAMMA2, gamma_m = None, gamma_f = None, gamma2_m = None, gamma2_f = None,
                 delta = DELTA, delta_m = None, delta_f = None,
                 epsilon = EPSILON, epsilon_m = None, epsilon_f = None, zeta = ZETA, zeta_m = None, zeta_f = None,
                 iota = IOTA, kappa = KAPPA, omega = OMEGA, phi = PHI, rho = RHO, tau = TAU,
                 shift_constant = SHIFT, decay_constant = DECAY, bump_constant = BUMP, 
                 min_adjacency = MIN_ADJ, max_adjacency = MAX_ADJ, min_investment = MIN_INV):
        self.r_function = r_function
        self.a_function = a_function
        
        self.alpha, self.alpha_m, self.alpha_f = (alpha, alpha_m, alpha_f)
        if alpha_m == None:
            self.alpha_m = alpha
        if alpha_f == None:
            self.alpha_f = alpha
       
        self.alpha2, self.alpha2_m, self.alpha2_f = (alpha2, alpha2_m, alpha2_f)
        if alpha2_m == None:
            self.alpha2_m = alpha2
        if alpha2_f == None:
            self.alpha2_f = alpha2
        
        self.aphas = alphas
        
        self.beta, self.beta_m, self.beta_f = (beta, beta_m, beta_f)
        if beta_m == None:
            self.beta_m = beta
        if beta_f == None:
            self.beta_f = beta
        
        self.beta2, self.beta2_m, self.beta2_f = (beta2, beta2_m, beta2_f)
        if beta2_m == None:
            self.beta2_m = beta2
        if beta2_f == None:
            self.beta2_f = beta2
        
        self.gamma2, self.gamma2_m, self.gamma2_f = (gamma2, gamma2_m, gamma2_f)
        if gamma2_f == None:
            self.gamma2_f = gamma2
        if gamma2_m == None:
            self.gamma2_m = gamma2
            
        self.gamma, self.gamma_m, self.gamma_f = (gamma, gamma_m, gamma_f)
        if gamma_f == None:
            self.gamma_f = gamma
        if gamma_m == None:
            self.gamma_m = gamma

        self.delta, self.delta_f, self.delta_m = (delta, delta_m, delta_f)
        if delta_m == None:
            self.delta_m = delta
        if delta_f == None:
            self.delta_f = delta

        self.epsilon, self.epsilon_m, self.epsilon_f = (epsilon, epsilon_m, epsilon_f)
        if epsilon_f == None:
            self.epsilon_f = epsilon
        if epsilon_m == None:
            self.epsilon_m = epsilon

        self.zeta, self.zeta_m, self.zeta_f = (zeta, zeta_m, zeta_f)
        if zeta_m == None:
            self.zeta_m = zeta
        if zeta_f == None:
            self.zeta_f = zeta
        
        self.iota = iota
        self.kappa = kappa
        self.omega = omega
        self.phi = phi
        self.rho = rho
        self.tau = tau
        self.shift_constant = shift_constant
        self.decay_constant = decay_constant
        self.bump_constant = bump_constant
        self.min_adjacency = min_adjacency
        self.max_adjacency = max_adjacency
        self.min_investment = min_investment
        self.strat_m = strat_m
        self.strat_f = strat_f
        self.strats_m = strats_m
        self.strats_f = strats_f
        self.res_m = res_m    # Resource limitation for a male
        self.res_f = res_f    # Resource limitations for a female (NOTE: This might be unnecessary also....)
        self.ress_m = ress_m
        self.ress_f = ress_f
        self.n_males = n_males    # Number of males per trial 
        self.n_females = n_females # Number of females per trial 
        self.q_male = q_male
        self.q_female = q_female
        self.q_males = q_males
        self.q_females = q_females
        self.n_turns = n_turns # Number of turns per trial
        self.n_trials = n_trials  # Number of trials per simulation
        
        if alphas == None:
            self.alphas = [self.alpha] * self.n_females
        if strats_f == None:
            self.strats_f = [self.strat_f] * self.n_females
        if strats_m == None:
            self.strats_m = [self.strat_m] * self.n_males
        if ress_f == None:
            self.ress_f = [self.res_f] * self.n_females
        if ress_m == None:
            self.ress_m = [self.res_m] * self.n_males
        if self.q_females == None:
            self.q_females = [self.q_female] * self.n_females
        if self.q_males == None:
            self.q_males = [self.q_male] * self.n_males
    
    # If, for some reason you want to set it all to 0, here you go
    def zero_reward(self):
        self.alpha, self.alpha_m, self.alpha_f = (0, 0, 0)
        self.apha2, self.alpha2_m, self.alpha2_f = (0, 0, 0)
        self.beta, self.beta_m, self.beta_f = (0, 0, 0)
        self.beta2, self.beta2_m, self.beta2_f = (0, 0, 0)
        self.gamma2, self.gamma2_m, self.gamma2_f = (0, 0, 0)
        self.gamma, self.gamma_m, self.gamma_f = (0, 0, 0)
            
            
# Object containing all the birds. Cute, eh? 
class Aviary(object):
    def __init__(self, params = Parameters()):
# Initialize some parameters:
        self.params = params
        self.n_males = params.n_males
        self.n_females = params.n_females
        self.strats_m = params.strats_m
        self.strats_f = params.strats_f
        self.ress_m = params.ress_m
        self.ress_f = params.ress_f
        self.q_males = params.q_males
        self.q_females = params.q_females
# Build the male and female lists in the aviary. 
        self.males = [Male_bird(num, params.strats_m[num], params.ress_m[num], params.q_males[num]) 
                      for num in range(params.n_males)]
        self.females = [Female_bird(num, params.strats_f[num], params.ress_f[num], params.alphas[num], params.q_females[num]) 
                        for num in range(params.n_females)]

        ## This is a big important function. It is how the aviary of birds responds every turn
    def respond(self,history):
        # Initialize Turn
        turn = Turn(history.current_turn,history.n_males,history.n_females)
        current_turn = history.current_turn
        # Save matrices to the turn
        ## I think this needs to be altered so that males and females respond seperately
        ## I believe I'm getting trapped in small loops where females are investing based on 1 turn previous
        turn.adjacency = self.update_adjacency(history, turn)
        turn.invest = self.update_invest(history, turn) 
        turn.reward = self.update_reward(history, turn)
        
        #turn.invest = self.update_mf(history, turn)
        
        return turn
        #return history

    def mrespond(self,history):
        
        for m in range(self.n_males):
            history.invest_matrix[history.current_turn] = self.males[m].respond(history)
        return invest

    def frespond(self,history):       
        for f in range(self.n_females):
            history.invest_matrix[history.current_turn] = self.females[f].respond(history)   
        return history.invest_matrix[history.current_turn]
    
    def update_adjacency(self,history,turn):
        
        current_adjacency = investment_to_adjacency(history, self.params)
        return current_adjacency
    
    def update_invest(self,history,turn):
        ## This provides both males and females an opportunity to return investment and competition
        ## For it to work, I need to make sure everything works cleanly
        previous_invest = history.invest_matrix[turn.n - 1]
        #male_invest = np.zeros(np.shape(previous_invest))
        #female_invest = np.zeros(np.shape(previous_invest))
        history.invest_matrix[turn.n] = previous_invest 
        for m in range(history.n_males):
            history.invest_matrix[turn.n] = self.males[m].respond(history)
        for f in range(history.n_females):
            history.invest_matrix[turn.n] = self.females[f].respond(history)
        #new_invest = male_invest + female_invest
        #history.invest_matrix[turn.n][history.invest_matrix[turn.n] == 0] = .001
        return history.invest_matrix[turn.n]
    
    def update_reward(self,history,turn):
        new_reward = adjacency_to_reward(history,turn, self.params)
        return new_reward
    
    def update_mf(self,history,turn):
        for m in range(history.n_males):
            history.invest_matrix[turn.n] = self.males[m].respond(history)
        history.adjacency_matrix[turn.n] = investment_to_adjacency(history, self.params)
        history.reward_matrix[turn.n] = self.update_reward(history, turn)
        for f in range(history.n_females):
            history.invest_matrix[turn.n] = self.females[f].respond(history)
        return history.invest_matrix[turn.n]
        
        
class Male_bird(object):
    def __init__(self, num, strategy = 'M0', resources = 1, quality = 1.0): #Convention: Males have even strategies, females odd. 
        self.num = num
        self.strategy = strategy
        self.resources = resources
        self.quality = quality

##      Seed self investment, and normalize for resources
#   Functions to adjust and get info. 
### NOTE: This is where males apply strategy, strategies are saved externally

    def respond(self,history):
        new_investment = SimStrategies.choose('M0', self.resources, history, self.num)
        #new_investment = working_strategy(self.resources, history, self.num)
        return new_investment

#Class of female cowbirds: 
#Includes: resources, response matrix, reward matrix
#Also includes choice function determining response
class Female_bird(object):
    def __init__(self, num, strategy = 'F0', resources = 1, alpha = ALPHA, quality = 1.0):
        self.num = num
        self.strategy = strategy
        self.resources = resources
        self.alpha = alpha
        self.quality = quality
##      Seed self  investment, NOTE that for females this is which males are investing in them, not vice versa
#   Functions to adjust and get info. 

    def respond(self,history):
        new_investment = SimStrategies.choose(self.strategy, self.resources, history, self.num)
        #new_investment = f_working_strategy(self.resources, history, self.num)
        return new_investment

#Array type object containing cowbirds, allowing cleverness: 
#Keep history of investment, reward for both males and females

#NOTE: I want to change this so that we have a full matrix of investment, where birds can invest in themselves, in same-sex birds, with negative investment, etc. This is a trivial change as it relates to history, but it will end up being quite a lot of changes down the line. 
class History(object):
    def __init__(self, params = Parameters()):
## Initialize the matrix for the whole sim (save a bunch of time)
        n_males = params.n_males
        n_females = params.n_females
        n_turns = params.n_turns
        self.adjacency_matrix = np.zeros([n_turns,n_males + n_females, n_males + n_females])
        self.adjacency_matrix[0,:n_males,n_males:] = np.random.rand(n_males,n_females)
        self.invest_matrix = np.zeros(np.shape(self.adjacency_matrix))
        self.reward_matrix = np.zeros(np.shape(self.adjacency_matrix))       
        self.n_turns = params.n_turns
        self.n_males = params.n_males
        self.n_females = params.n_females
        self.current_turn = 0
        self.quality_males = params.q_males
        self.quality_females = params.q_females
        self.quality_vector = np.append(params.q_males,params.q_females)
        self.params = params

    def record(self,turn): 
        self.adjacency_matrix[turn.n] = turn.adjacency
        self.invest_matrix[turn.n] = turn.invest
        self.reward_matrix[turn.n] = turn.reward
        self.advance()
        
    def initialize(self,initial_conditions = None): #set first investment conditions
        
        if initial_conditions == None:
            self.invest_matrix[0] = np.random.random([self.n_males + self.n_females, self.n_males + self.n_females]) * (1 - np.identity(self.n_males + self.n_females))
            self.invest_matrix[0] = self.invest_matrix[0] / self.invest_matrix[0].sum(1)
            self.adjacency_matrix[0] = np.random.random(np.shape(self.invest_matrix[0])) * (1 - np.identity(self.n_males + self.n_females))
            #Inititalize the first reward is a little clunky, because you need a turn...
            turn0 = Turn(n=0, n_males = self.n_males, n_females = self.n_females)
            turn0.invest = self.invest_matrix[0]
            turn0.adjacency = self.adjacency_matrix[0]
            turn0.n = 0
            self.reward_matrix[0] = adjacency_to_reward(self, turn0, self.params)
        else:
            self.invest_matirx[0] = initial_conditions.invest
            self.reward_matrix[0] = initial_conditions.reward
            self.adjacency_matrix[0] = initial_conditions.adjacency
        self.reward_matrix[-1] = self.reward_matrix[0] # This is for when the f_respond checks for previous
        self.invest_matrix[-1] = self.invest_matrix[0]
        self.adjacency_matrix[-1] = self.adjacency_matrix[0]       
        
    def advance(self):
        self.current_turn = self.current_turn + 1       

# This is perhaps redundant, it just exists to pass in a first turn, but the first turn might do that perfectly well...
class Conditions(object):
    def __init__(self, history, t):
        self.invest = history.invest_matrix[t]
        self.reward = history.reward_matrix[t]
        self.adjacency = history.adjacency_matrix[t]
        
#Object containing turn
class Turn(object):
    def __init__(self, n, n_males, n_females, previous_turn = None):
## Initialize based on last turn if desired, otherwise start blank
        self.n = n
        if previous_turn != None:
            self.invest = previous_turn.invest
            self.reward = previous_turn.reward
            self.adjacency = previous_turn.adjacency
            self.invest = previous_turn.invest
            self.reward = previous_turn.reward
        else:
            self.invest = np.zeros([n_males,n_females])
            self.reward = np.zeros([n_males,n_females])
            self.adjacency = np.zeros([n_males+n_females,n_females+n_males])
            self.invest = np.zeros(np.shape(self.adjacency))
            self.reward = np.zeros(np.shape(self.adjacency))

#Functions determining success: 
#This is not technically important for the simulation, but it will determine which strategy is best, which is important

def female_success(params):

    return success

def male_success(params):

    return success

def run_trial(params = Parameters(), initial_conditions = None):
    n_males = params.n_males
    n_females = params.n_females
    n_turns = params.n_turns
## Initialize full record...
## initialize history
    history = History(params)
    history.initialize(initial_conditions)
# Build an aviary using the given parameters
    aviary = Aviary(params)
# Initialize initial investment (based on resources, if specified)
    history.initialize()
    history.advance()
# For every turn, calculate response and record it in history.
    for t in range(n_turns-1):
        turn = aviary.respond(history)
        history.record(turn)
    return history
       
def run_simulation(params = Parameters()):

    record = [0 for tr in range(params.n_trials)]
    for tr in range(params.n_trials):
        history = run_trial(params)
        record[tr] = SimStats.get_stats(history) 
# For tidiness, stats is saved in a seperate file
    return record

#### Plotting Functions ####
    
def plot_history(history_matrix):
    # generate a five layer data 
    data = history_matrix
    # current layer index start with the first layer 
    idx = 0

    # figure axis setup 
    fig, ax = plt.subplots()
    fig.subplots_adjust(bottom=0.15)

    # display initial image 
    im_h = ax.imshow(data[idx, :, :], vmin=-1.0, vmax=1., cmap='bwr', interpolation='nearest')

    # setup a slider axis and the Slider
    ax_depth = plt.axes([0.23, 0.02, 0.56, 0.04])
    slider_depth = Slider(ax_depth, 'depth', 0, data.shape[0]-1, valinit=idx)

    # update the figure with a change on the slider 
    def update_depth(val):
        idx = int(round(slider_depth.val))
        im_h.set_data(data[idx, :, :])

    slider_depth.on_changed(update_depth)

    plt.show()
    
### Several Functions for Network Plotting ###

def plot_network_progression(history):
    turns = history.current_turn
    seed_turn = 0
    adj_rounded = np.round(history.adjacency_matrix[seed_turn] + .4) * 4
    G = nx.from_numpy_matrix(adj_rounded)
    
    pos = nx.spring_layout(G)
    step = turns / 20
    wait_time = .5
    for t in range(0,turns,step):
        plt.cla()
        adj_rounded = history.adjacency_matrix[t]
        adj_rounded[adj_rounded < .15] = 0.0
        adj_rounded.dtype = [('weight','float')]
        G = nx.from_numpy_matrix(adj_rounded,parallel_edges=False)
        pos = nx.spring_layout(G, pos=pos, fixed=None)
        color_map = []
        for node in G:
            if int(node) < history.n_males:
                color_map.append('blue')
            else: color_map.append('red')    
        edges = G.edges()
        weights = [G[u][v]['weight'] for u,v in edges]
        nx.draw(G, pos, node_color = color_map, with_labels = True, edges=edges, width=weights, node_size=450)
        plt.pause(wait_time)
        plt.draw()
    
def plot_network(adjacency):
    plt.cla()
    G = nx.from_numpy_matrix(adjacency,parallel_edges=False)
    # Set layount: 
    pos = nx.spring_layout(G)
    color_map = []
    n_males, n_females = np.shape(investment)
    for node in G:
        if int(node) < n_males:
            color_map.append('blue')
        else: color_map.append('red')    

    edges = G.edges()
    weights = [G[u][v]['weight'] for u,v in edges]
    nx.draw(G, pos, node_color = color_map, with_labels = True, edges=edges, width=weights)

    plt.show()
        

if __name__ == "__main__":
    
    #history = run_trial()
    #run_simulation(alphas = [1.5]*N_FEMALES)
    #plot_history(history)
    #plot_network_progression(history)
    pass

In [2]:
## The min investment condition turns out to be important. 
# It basically forces a 'best of -n' freeze out of the best, as non-investment drops out
# If there is constant minimum investment, they tend towards equilibrium as the average is low.
# This is, perhaps, an artifact of the way it's normalized
# Regardless, this can be overcome by amping up the response curve, 
# such that majority commitment yields huge returns
# All this assumes that females have no input in the matter

params = Parameters(zeta = 0)
params.zero_reward()
params.min_investment = 0.000
params.beta2_m = 5
params.alpha_m = 1.0
params.beta_f = 1.0
params.strat_m = 'M3'
params.strat_f = 'F3'
params.n_turns = 800
params.decay_constant = 0.05
history = run_trial(params)
plot_history(history.invest_matrix)
#plot_network_progression(history)

In [7]:
plot_network_progression(history)

In [ ]:
params = Parameters(zeta = 0)
params.zero_reward()
#params.min_investment = 0.001
#params.min_adjacency = 0.001
params.alpha2_m = 1.0
params.gamma_m = .1
params.beta_f = 50.0
params.shift_constant = .2
#params.alpha_f = 1.0
params.strat_m = 'M0'
params.strat_f = 'F0'
params.n_turns = 800
params.decay_constant = 0.1
history = run_trial(params)
plot_history(history.invest_matrix)
#plot_network_progression(history)
plot_history(history.adjacency_matrix)

In [5]:
plot_history(history.adjacency_matrix)

In [2]:
# Fairly Default trial: 
params = Parameters(alpha = 0, alpha2_m = 1.5, alpha_f = 1.0, alpha2_f = 0, beta_m = 0, beta_f = 0, gamma=0, delta=0, 
                    epsilon=0, zeta=0, bump_constant=0.0, min_investment=0, 
                    min_adjacency=0, decay_constant=0.01, n_turns=800)
params.strat_m = 'M3'
params.strat_f = 'F3'
history = run_trial(params)
plot_history(history.invest_matrix)
#plot_history(history.reward_matrix)
#plot_history(history.adjacency_matrix)
plot_network_progression(history)

/Users/Ammon/anaconda/lib/python2.7/site-packages/matplotlib/collections.py:650: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

/Users/Ammon/anaconda/lib/python2.7/site-packages/matplotlib/collections.py:590: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



In [11]:
plot_history(history.reward_matrix / history.invest_matrix)

In [50]:
a_list = np.arange(.5,2,.1)
record_means = np.array([])
record_stds = np.array([])

for a in a_list:
    print 'running for ' + str(a)
    params = Parameters(alpha = a, beta=1.0, gamma = 1, bump_constant=0.0001, min_adjacency=0, min_investment=0,decay_constant=0.01, n_turns=800)
    record = run_simulation(params)

    ms = np.zeros(len(record))
    for m in range(len(record)):
        ms[m] = record[m]['monogamy']

    record_means = np.append(record_means, ms.mean())
    record_stds = np.append(record_stds, ms.std())

subprocess.call(["afplay","/System/Library/Sounds/Submarine.aiff"])    
plt.errorbar(a_list, record_means, record_stds)
plt.show()


running for 0.5


KeyboardInterrupt: 

In [44]:
import subprocess
subprocess.call(["afplay","/System/Library/Sounds/Submarine.aiff"])

0

In [29]:
ms = np.zeros(len(record))
for m in range(len(record)):
    ms[m] = record[m]['monogamy']

print ms.mean()
print ms.std()

0.792857142857
0.220041740011


In [84]:
params = Parameters(alpha = 1.0, bump_constant=0, n_turns=300)
history = run_trial(params)
plot_history(history.invest_matrix)
plot_history(history.reward_matrix)
plot_history(history.adjacency_matrix)

In [10]:
def working_strat_f(resources, history, num):
    shift_scaler = .01
    n_males = history.n_males
    current_turn = history.current_turn
    
    current_invest = history.invest_matrix[current_turn]
    
    # Make Competition Decisions
    compete = compete_flee_f(resources, history, num)
    # Balance Resources
    comp_res = sum(np.abs(compete))
    comp_cost = get_cost_f(history, num)
    mate_resources = resources - comp_cost - comp_res
    
    ## Mate with males
    my_previous_reward = np.zeros(np.shape(history.reward_matrix[current_turn - 1,history.n_males + num,:]))
    my_previous_invest = np.zeros(np.shape(my_previous_reward))

    my_previous_reward[:] = history.reward_matrix[current_turn - 1,history.n_males + num,:]
    my_previous_invest[:] = history.invest_matrix[current_turn - 1,history.n_males + num,:]
    
    ## Force old female investment to 0 to avoid skewing avg
    my_previous_invest[n_males:] = 0
    my_previous_reward[n_males:] = 0                             
    
    my_current_invest = np.zeros(np.shape(my_previous_invest))
    
    profit = my_previous_reward[:] / (my_previous_invest[:] + .0001)

    avg_profit = profit[my_previous_invest > 0.0].mean()
    
    #shift_matrix = np.zeros(np.shape(profit))    
    #shift_matrix[profit > avg_profit] = shift_scaler
    
    #shift_matrix[profit < avg_profit] = -shift_scaler
    shift_matrix = profit - avg_profit
    from IPython.core.debugger import Tracer; Tracer()()
    #my_current_invest = my_previous_invest + shift_matrix
    my_current_invest = my_previous_invest + shift_matrix
    my_current_invest[my_current_invest < 0] = history.params.min_investment

    
    my_current_invest = my_current_invest / (sum(abs(my_current_invest)) +.0001) * mate_resources 
    my_current_invest[history.n_males:] = compete
        
    current_invest[history.n_males + num,:] = my_current_invest   
    return current_invest   

def get_cost_f(history,num):
    a = 1.0
    current_turn = history.current_turn
    previous_invest = history.invest_matrix[current_turn - 1]
    n_males = history.n_males
    n_females = history.n_females
    cost = [0] * n_females
    j = num + history.n_males
    for f in range(n_females):
        i = f + n_males
        cost[f] = (max(previous_invest[i,j],0) + min(previous_invest[j,i],0)) * history.adjacency_matrix[current_turn,j,i] ** a 
    return sum(cost)

def compete_flee_f(resources,history,num):
    compete = [0] * history.n_females
    previous_invest = history.invest_matrix[history.current_turn - 1]
    # Run from any competition
    j = num + history.n_males
    for f in range(history.n_females):
        i = f + history.n_males
        compete[f] = -1 * max(0,previous_invest[i,j])    
    # Make sure you're not going into negatives
    if sum(compete) > (resources / 2.0):
        compete = compete / sum(compete) * (resources / 2.0)
    return compete

In [11]:
history.current_turn = 799
response = working_strat_f(1.0, history, 1)
print np.round(response, 3)

> <ipython-input-10-a5d61795fc0c>(39)working_strat_f()
     37     from IPython.core.debugger import Tracer; Tracer()()
     38     #my_current_invest = my_previous_invest + shift_matrix
---> 39     my_current_invest = my_previous_invest + shift_matrix
     40     my_current_invest[my_current_invest < 0] = history.params.min_investment
     41 

ipdb> shift_matrix
array([ 0.01528941,  0.01141085,  0.01537753, -0.02084099, -0.02123681,
        1.42301824, -0.61939933, -0.61939933, -0.61939933, -0.61939933,
       -0.61939933, -0.61939933])
ipdb> n
> <ipython-input-10-a5d61795fc0c>(40)working_strat_f()
     38     #my_current_invest = my_previous_invest + shift_matrix
     39     my_current_invest = my_previous_invest + shift_matrix
---> 40     my_current_invest[my_current_invest < 0] = history.params.min_investment
     41 
     42 

ipdb> n
> <ipython-input-10-a5d61795fc0c>(43)working_strat_f()
     41 
     42 
---> 43     my_current_invest = my_current_invest / (sum(abs(my_current_in

In [21]:
params.shift_constant

0.1

In [33]:
params.zero_reward()

In [34]:
print params.alpha

0


In [36]:
params.__init__()

In [37]:
print params.alpha

1.5
